<h1 style="color: skyblue"> Deeptech Engineer Intern Challenge </h1>

In [1]:
# imports
import pandas as pd
from fuzzywuzzy import process
import random

/home/marius/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Facebook dataset
chunk_size = 1000000 
facebook_dataset = pd.read_csv('facebook_dataset.csv', delimiter=',', quotechar='"', on_bad_lines='skip')
facebook_dataset.head(5)

,domain,address,categories,city,country_code,country_name,description,email,link,name,page_type,phone,phone_country_code,region_code,region_name,zip_code
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",NaN,calvisson,fr,france,NaN,NaN,https://euro-hygiene-34.fr,Euro Hygiène,LocalBusiness,NaN,NaN,occ,occitanie,30420
1,lakesidehomeservices.ca,NaN,Appliance Repair & Maintenance|Home Builders &...,NaN,ca,NaN,NaN,NaN,https://lakesidehomeservices.ca,Lakeside Home Services,LocalBusiness,1.613832e+10,ca,NaN,NaN,NaN
2,rossiterboats.com,NaN,Boats & Yachts Dealers|Boat Tours & Cruises,NaN,NaN,NaN,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,Rossiter Boats,LocalBusiness,NaN,NaN,NaN,NaN,NaN
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...",Architects & Architectural Services|Other Engi...,vancouver,ca,canada,NaN,office@caa-arcitecture.com,http://caa-architecture.com,Chandler Associates Architecture Inc.,LocalBusiness,1.604687e+10,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",NaN,winnipeg,ca,canada,NaN,NaN,https://apexsurety.ca,Apex Surety & Insurance Ltd.,LocalBusiness,1.204560e+10,ca,mb,manitoba,r3l 2t7


In [3]:
# Google dataset

google_dataset = pd.read_csv('google_dataset.csv', delimiter=',', quotechar='"', on_bad_lines='skip')
google_dataset.head(5)

/tmp/ipykernel_6365/98803827.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  google_dataset = pd.read_csv('google_dataset.csv', delimiter=',', quotechar='"', on_bad_lines='skip')


,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain
0,"28 Central Coast Hwy, West Gosford NSW 2250, A...",Fabric-Based Home Goods,gosford,au,australia,Spotlight West Gosford,+61243355946,au,"West Gosford NSW, Australia",+61 2 4335 5946,nsw,new south wales,"4.1 (766) · Craft store West Gosford NSW, Aust...",2250,spotlightstores.com
1,"400 Scott St, St. Catharines, ON L2M 3W2, Canada",Book Stores,st. catharines,ca,canada,Heritage Christian Book Store,+19059374553,ca,400 Scott St · In Grantham Plaza,+1 905-937-4553,on,ontario,4.7 (100) · Book store 400 Scott St · In Grant...,l2m 3w2,bookmanager.com
2,"191 Pleasant St, Yarmouth, NS B5A 2J9, Canada",Other Building Material Retailers,yarmouth,ca,canada,Pleasant Timber Mart,+19027429181,ca,"7+ years in business · Yarmouth, NS, Canada",+1 902-742-9181,ns,nova scotia,4.7 (40) · Building materials store 7+ years i...,b5a 2j9,timbermart.ca
3,"11040 Santa Monica Blvd Suite 370, Los Angeles...",Plastic Surgery Clinics,los angeles,us,united states,Skin Specifics Medical Spa West LA,+18184268353,us,7+ years in business · 11040 Santa Monica Blvd...,+1 818-426-8353,ca,california,4.3 (15) · Medical spa 7+ years in business · ...,90025,linktr.ee
4,NaN,Catering & Delivery,NaN,NaN,NaN,Lakay express,NaN,NaN,NaN,NaN,NaN,NaN,5.0 (1) · Mobile caterer Open ⋅ Closes 9PM,NaN,linktr.ee


In [4]:
# Website dataset

website_dataset = pd.read_csv('website_dataset.csv', delimiter=';', quotechar='"', on_bad_lines='skip')
website_dataset.head(5)

,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting


In [5]:
# common columns

columns_facebook = set(facebook_dataset.columns)
columns_google = set(google_dataset.columns)
columns_website = set(website_dataset.columns)


# function to find similar columns
def find_similar_columns(columns1, columns2, columns3):
    similar_columns = []
    for col1 in columns1:
        # find the most similar column in columns2 and columns3
        match2 = process.extractOne(col1, columns2)
        match3 = process.extractOne(col1, columns3)
        # consider a match if similarity score is above a threshold (80)
        if match2[1] > 80 and match3[1] > 80:
            similar_columns.append((col1, match2[0], match3[0]))
    return similar_columns

# find similar columns across the datasets
similar_columns = find_similar_columns(columns_facebook, columns_google, columns_website)

# print the similar columns
for col1, col2, col3 in similar_columns:
    print(f"Similar Columns: {col1}, {col2}, {col3}")

Similar Columns: phone, phone, phone
Similar Columns: domain, domain, domain_suffix
Similar Columns: name, name, legal_name
Similar Columns: phone_country_code, phone_country_code, phone
Similar Columns: city, city, main_city


In [6]:
# modify root_domain with doman in website dataset
website_dataset.rename(columns = {'root_domain': 'domain'}, inplace = True)
website_dataset.head(5)

,domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting


In [7]:
# make a new DataFrame

merged_df = pd.merge(facebook_dataset['domain'], google_dataset['domain'], on = 'domain', how = 'outer')
merged_df = pd.merge(merged_df, website_dataset['domain'], on = 'domain', how = 'outer')

# drop duplicates
merged_df.drop_duplicates(inplace = True)

# make domain as primary key / id
merged_df.set_index('domain', inplace = True)
merged_df.sample(5)

""
domain
advancedaluminum.ca
arquitectalauragalleano.com
allodestinations.com
meigslibrary.org
thereiteclub.com


In [8]:
# add to dataframe following common columns: Category, Address, Phone, Company names.

# first preprocess the categories to have the same column name

# category column name
facebook_dataset.rename(columns = {'categories': 'category'}, inplace = True)
website_dataset.rename(columns = {'s_category': 'category'}, inplace = True)

In [9]:
facebook_dataset['category'].sample(5)

60578    Nursing & Personal Care|Senior Care and Assist...
5685                       Decorators & Interior Designers
40290    Community Center|Other schools|Elementary Schools
66832                                                  NaN
68632              General Contractors|Roofing Contractors
Name: category, dtype: object

In [10]:
website_dataset['category'].sample(5)

6184     Meat, Fish & Seafood Stores
8491               Bed and Breakfast
15178              Religious Schools
29563         Real Estate Investment
53578             Indian Restaurants
Name: category, dtype: object

In [11]:
# address is missing from website dataset

# phone is the same

# website dataset is different, should be name
website_dataset.rename(columns = {'legal_name': 'name'}, inplace = True)
website_dataset['name'].sample(5)

18227                              NaN
69126      Europlex intranational Ltd.
30824                              NaN
68608             Expert Electric Ltd.
13482    VITRES TEINTEES BLACK FOX INC
Name: name, dtype: object

In [12]:
# drop duplicates from all datasets
website_dataset.drop_duplicates(inplace = True)
facebook_dataset.drop_duplicates(inplace = True)
google_dataset.drop_duplicates(inplace = True)

In [13]:
# merge category column
merged_category = pd.merge(facebook_dataset[['domain', 'category']], google_dataset[['domain', 'category']], on = "domain")
merged_category = pd.merge(merged_category, website_dataset[['domain', 'category']], on = "domain")
merged_category.rename(columns = {'category': 'category_z'}, inplace = True)
merged_category.drop_duplicates(inplace  = True)
merged_category.sample(5)

,domain,category_x,category_y,category_z
153636,stewartproperties.ca,Property Management|Real Estate Developers,Property Management,Real Estate - Agents & Managers
53274,accessinsurancegroup.com,Banks & ATMs|Insurance - Agents|Insurance - ...,"Insurance - Agents, Carriers & Brokers",Auto Insurance Agency
323819,calgarywestdance.com,Dance Schools|Music Schools & Instructors,Dance Studios & Schools,Dance Schools
185395,amherstcremation.com,Funeral Services & Cemeteries,Pet Cemeteries,Funeral Services & Cemeteries
6441,crocketttexas.org,Fire Departments & Fire Protection,"Executive, Legislative & Government Institutions","Executive, Legislative & Government Institutions"


In [14]:
# merge address column
merged_address = pd.merge(facebook_dataset[['domain', 'address']], google_dataset[['domain', 'address']], on = "domain")
merged_address.rename(columns = {'address': 'address_z'}, inplace = True)
merged_address.drop_duplicates(inplace  = True)
merged_address.sample(5)

,domain,address_x,address_y
280372,ikea.com,"1 hollinswsorth road, 2765, marsden park, nsw,...","Ulitsa Timofeya Charkova, 60, Tyumen, Tyumen O..."
311389,postoffice.co.uk,"253 lodge causeway, fishponds, bs163, bristol,...","66-68 John St, Wimblebury, Cannock WS12 2RJ, U..."
84421,mdanderson.org,"1515 holcombe blvd., 77030, houston, tx, texas","6900 Fannin St, Houston, TX 77030"
101491,sehamsboutique.com,NaN,NaN
63839,lakesideschool.org,"14050 1st ave ne, 98125, seattle, wa, washington","14050 1st Ave NE, Seattle, WA 98125"


In [15]:
# concat category and address into a new dataframe
# DF = pd.merge(merged_category, merged_address, on = 'domain')
# DF.drop_duplicates(inplace = True)
# DF.head(5)
DF = pd.concat([merged_category, merged_address], axis = 0)
DF.head(5)

,domain,category_x,category_y,category_z,address_x,address_y
0,euro-hygiene-34.fr,NaN,Pest Services & Agricultural Chemicals,Pest Services & Agricultural Chemicals,NaN,NaN
1,lakesidehomeservices.ca,Appliance Repair & Maintenance|Home Builders &...,Appliance Repair & Maintenance,Appliance Repair & Maintenance,NaN,NaN
2,rossiterboats.com,Boats & Yachts Dealers|Boat Tours & Cruises,Boats & Yachts Dealers,Boats & Yachts Repair Shops,NaN,NaN
3,caa-architecture.com,Architects & Architectural Services|Other Engi...,Architects & Architectural Services,Architects & Architectural Services,NaN,NaN
4,apexsurety.ca,NaN,"Insurance - Agents, Carriers & Brokers",Liability Insurance Agency,NaN,NaN


In [16]:
# merge phone column
merged_phone = pd.merge(facebook_dataset[['domain', 'phone']], google_dataset[['domain', 'phone']], on = "domain")
merged_phone = pd.merge(merged_phone, website_dataset[['domain', 'phone']], on = "domain")
merged_phone.rename(columns = {'phone': 'phone_z'}, inplace = True)
merged_phone.drop_duplicates(inplace  = True)
merged_phone.sample(5)

,domain,phone_x,phone_y,phone_z
219942,facebook.com,NaN,18644767967.0,18772765554
14289,pharmasave.com,NaN,17094442202.0,16044552400
64587,bouclair.com,NaN,+14186288760,18002682524
245770,facebook.com,NaN,6435777888.0,18772765554
338513,lasergame-evolution.com,3.354643e+10,33185460111.0,33476130741


In [17]:
# concat DF and phone into a new dataframe
DF = pd.concat([DF, merged_phone], axis = 0)
DF.head(5)

,domain,category_x,category_y,category_z,address_x,address_y,phone_x,phone_y,phone_z
0,euro-hygiene-34.fr,NaN,Pest Services & Agricultural Chemicals,Pest Services & Agricultural Chemicals,NaN,NaN,NaN,NaN,NaN
1,lakesidehomeservices.ca,Appliance Repair & Maintenance|Home Builders &...,Appliance Repair & Maintenance,Appliance Repair & Maintenance,NaN,NaN,NaN,NaN,NaN
2,rossiterboats.com,Boats & Yachts Dealers|Boat Tours & Cruises,Boats & Yachts Dealers,Boats & Yachts Repair Shops,NaN,NaN,NaN,NaN,NaN
3,caa-architecture.com,Architects & Architectural Services|Other Engi...,Architects & Architectural Services,Architects & Architectural Services,NaN,NaN,NaN,NaN,NaN
4,apexsurety.ca,NaN,"Insurance - Agents, Carriers & Brokers",Liability Insurance Agency,NaN,NaN,NaN,NaN,NaN


In [18]:
# merge company column
merged_company = pd.merge(facebook_dataset[['domain', 'name']], google_dataset[['domain', 'name']], on = "domain")
merged_company = pd.merge(merged_company, website_dataset[['domain', 'name']], on = "domain")
merged_company.rename(columns = {'name': 'name_z'}, inplace = True)
merged_company.drop_duplicates(inplace  = True)
merged_company.sample(5)

,domain,name_x,name_y,name_z
261017,facebook.com,Facebook,CircusYoga kelowna,Facebook Ireland Ltd.
105003,pennmedicine.org,Penn Medicine,"Rachel Kelz, MD","A.D.A.M., Inc."
5968,moreautomation.com,More Automation Solutions,More Automation Solutions Inc.,More Automation Solutions Inc.
200818,facebook.com,Facebook,Stevens Packing,Facebook Ireland Ltd.
198343,facebook.com,Facebook,Nyquist Roundhouse Martial Arts,Facebook Ireland Ltd.


In [19]:
# concat DF and name into a new dataframe
DF = pd.concat([DF, merged_company], axis = 0)
DF.sample(10)

,domain,category_x,category_y,category_z,address_x,address_y,phone_x,phone_y,phone_z,name_x,name_y,name_z
36607,marykay.com,NaN,NaN,NaN,NaN,"30 Harold St, Roxbury, MA 02119",NaN,NaN,NaN,NaN,NaN,NaN
207102,facebook.com,NaN,NaN,NaN,NaN,NaN,NaN,+442380616484,18772765554,NaN,NaN,NaN
305683,cityofsacramento.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"City of Sacramento, Government",Oak Park Frisbee Golf Course,"Sacramento Heritage, Inc."
42744,conad.it,NaN,NaN,NaN,NaN,NaN,NaN,+390432665302,3.90E+12,NaN,NaN,NaN
214108,facebook.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Facebook,Salon Romanesc,Facebook Ireland Ltd.
183026,naganoskate.com,Malls & Shopping Center|Sporting Goods Store,Industrial Machinery & Supplies,Sporting Goods Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172312,oracle.com,Business Consulting|Computer Stores,Courthouses,Telecommunications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335704,instagram.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Dragonfly,AFZ-Autos Ltd,NaN
69795,doctolib.fr,NaN,NaN,NaN,NaN,NaN,NaN,+33373720177,33143172211,NaN,NaN,NaN
181917,airbnb.com,NaN,NaN,NaN,NaN,NaN,NaN,18037612980.0,18504871395,NaN,NaN,NaN


In [20]:
# It first checks if there is only one non-NaN value in the category columns. If so, it returns that value. 
# If all category columns have the same value, it returns that value. If there are multiple non-NaN values but they share a common word, 
# it returns the value containing that common word. If none of the above conditions are met, it returns a random non-NaN value from the category columns

# function to check for common words

def common_words(row, features):
    for cat in features:
        for other_cat in features:
            if cat != other_cat:
                val1 = str(row[cat]) if not pd.isnull(row[cat]) else ''
                val2 = str(row[other_cat]) if not pd.isnull(row[other_cat]) else ''
                if val1 and val2:  # Check if both values are non-empty
                    common = set(val1.split()).intersection(val2.split())
                    if common:
                        return common.pop()  # return the common word
    return None

# function to assign category based on conditions
def assign_feature(row, categories):
    values = [row[cat] for cat in categories]
    non_nan_values = [val for val in values if not pd.isnull(val)]
    
    if len(non_nan_values) == 1:
        return non_nan_values[0]
    elif len(set(values)) == 1:  # all categories have the same value
        return values[0]
    elif non_nan_values:
        common_word = common_words(row, categories)
        if common_word:
            for cat in categories:
                if common_word in str(row[cat]).split():
                    return row[cat]
        else:
            for cat in categories:
                if values.count(row[cat]) > 1:  # duplicate values in other categories
                    return row[cat]
    else:
        non_nan_values = [val for val in row[categories] if not pd.isnull(val)]
        if non_nan_values:
            return random.choice(non_nan_values)
        else:
            return None

In [21]:
category_cols = ['category_x', 'category_y', 'category_z']
address_cols = ['address_x', 'address_y']
phone_cols = ['phone_x', 'phone_y', 'phone_z']
name_cols = ['name_x', 'name_y', 'name_z']

DF['category'] = DF.apply(lambda row: assign_feature(row, category_cols), axis=1)
DF['address'] = DF.apply(lambda row: assign_feature(row, address_cols), axis=1)
DF['phone'] = DF.apply(lambda row: assign_feature(row, phone_cols), axis=1)
DF['name'] = DF.apply(lambda row: assign_feature(row, name_cols), axis=1)

DF.sample(10)

,domain,category_x,category_y,category_z,address_x,address_y,phone_x,phone_y,phone_z,name_x,name_y,name_z,category,address,phone,name
308875,postoffice.co.uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mayfield Park Post Office,Hills Road Post Office,Post Office Limited,NaN,NaN,None,Mayfield Park Post Office
141627,uhaul.com,NaN,NaN,NaN,"487 rte 132, g4w 7c5, matane, qc, canada, quebec","201 4th St, Bakersfield, CA 93304",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
141141,uhaul.com,NaN,NaN,NaN,NaN,NaN,1.418567e+10,16049964088.0,18004684285,NaN,NaN,NaN,NaN,NaN,None,NaN
79047,relaxzwembaden.be,NaN,NaN,NaN,"retiesebaan 131a, 2460, kasterlee, belgium, fl...","131A Unit 5, Retiesebaan, 2460 Kasterlee, Belgium",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
66952,intertek.com,NaN,NaN,NaN,NaN,NaN,NaN,94112877300.0,4.97E+12,NaN,NaN,NaN,NaN,NaN,None,NaN
216136,facebook.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Facebook,Signs For Less,Facebook Ireland Ltd.,NaN,NaN,None,Facebook
246397,facebook.com,NaN,NaN,NaN,NaN,NaN,NaN,18147553691.0,18772765554,NaN,NaN,NaN,NaN,NaN,None,NaN
207644,facebook.com,NaN,NaN,NaN,NaN,"3205 Valentine Rd, North Little Rock, AR 72117",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3205 Valentine Rd, North Little Rock, AR 72117",None,NaN
34527,ihg.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,voco St. David's Cardiff,"Holiday Inn Express & Suites Carpinteria, an I...",IHG Hotels Limited.,NaN,NaN,None,"Holiday Inn Express & Suites Carpinteria, an I..."
30550,smartgardian.com,Security Guards & Patrol Services|Security Gua...,"Alarms, Surveillance & Security Systems","Alarms, Surveillance & Security Systems",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Security Guards & Patrol Services|Security Gua...,NaN,None,NaN


In [22]:
DF.drop(columns = ['category_x', 'category_y', 'category_z', 'address_x', 'address_y', 'phone_x', 'phone_y', 'phone_z', 'name_x', 'name_y', 'name_z'], axis = 1, inplace = True)

In [36]:
# final DF
DF.rename(columns = {'name': 'company name'}, inplace = True)
DF = DF[['domain', 'company name', 'category', 'address', 'phone']]
DF.sample(10)

,domain,company name,category,address,phone
318891,exploretock.com,NaN,NaN,"1235 26 Ave SE, Calgary, AB T2G 1R7, Canada",None
15932,uwhealth.org,None,NaN,NaN,None
142053,accessmortgagecanada.com,NaN,NaN,None,None
214935,facebook.com,NaN,NaN,"25 Rue de l'Aige au Carabin, 21560 Arc-sur-Til...",None
225257,facebook.com,Facebook,NaN,NaN,None
324750,thedressingagency.com,NaN,NaN,"69 wingold ave, m6b1p8, toronto, on, canada, o...",None
127182,booking.com,NaN,NaN,NaN,None
320808,ontarioparks.com,NaN,NaN,"Sharbot Lake, ON, Canada",None
274426,mitre10.co.nz,Mitre 10 New Zealand,NaN,NaN,None
155406,yell.com,NaN,NaN,None,None
